# From Heuristics to Plausible Models: Deconstructing Transformer Attention with Sparse Distributed Memory

**An Interactive Exploration of "Attention Approximates Sparse Distributed Memory"**

This Jupyter notebook serves as an in-depth, interactive educational guide to the research presented in the lecture, which demonstrates that the highly successful but heuristic **Transformer Attention** mechanism can be understood as a close approximation of **Sparse Distributed Memory (SDM)**. SDM is a model of associative memory developed by Pentti Kanerva in 1988, notable for its mathematical properties and biological plausibility.

We will deconstruct the core concepts of both SDM and Attention, implement them from scratch, and reproduce the central claim of the research: the exponential decay in the geometry of high-dimensional spaces, which forms the foundation of SDM's retrieval mechanism, is functionally equivalent to the softmax function in attention.

### Section 1: Overview & Prerequisites

#### **1.1 Summary of the Research Topic**

The lecture posits that the heuristic softmax operation, a critical component of the Transformer architecture's attention mechanism, is not just a statistical convenience but has a deeper connection to the principles of associative memory. Specifically, it shows that the attention mechanism approximates the read operation of a Sparse Distributed Memory. This connection is established by demonstrating that the number of shared "neurons" (or vectors) in the intersection of two hyperspheres in a high-dimensional space—the core of SDM's retrieval process—decays approximately exponentially as the hyperspheres move apart. This exponential relationship mirrors the behavior of the softmax function, providing a theoretical and biologically plausible foundation for one of deep learning's most powerful tools. The lecture further suggests this link could explain why Transformers are so effective and points to the cerebellum as a potential site for a similar mechanism in the brain.

#### **1.2 Prerequisite Knowledge**

**Mathematical Concepts:**
- **Linear Algebra:** Vectors, vector spaces (especially high-dimensional), dot products, matrix multiplication.
- **Geometry in High Dimensions:** Understanding of hyperspheres and distance metrics (Hamming Distance, Cosine Similarity).
- **Calculus & Probability:** The exponential function, the softmax function, and the concept of a weighted average.

**Machine Learning / Computer Science Concepts:**
- **Associative Memory:** The general concept of content-addressable memory (e.g., Hopfield Networks).
- **Neural Networks:** Basic understanding of neurons, layers, and activation functions.
- **Transformers:** Familiarity with the Query, Key, and Value architecture of the attention mechanism.

#### **1.3 Hierarchy of Topics**

1.  **Mathematical Foundations:** We'll start by implementing the core mathematical tools: distance metrics and the softmax function.
2.  **Sparse Distributed Memory (SDM):** A from-scratch implementation and conceptual overview of this associative memory model.
3.  **Transformer Attention:** A brief implementation of the standard attention mechanism for comparison.
4.  **The Core Research: Attention as SDM:** We will connect the two by visualizing and proving the exponential approximation at the heart of the research.
5.  **Experimental Analysis:** We will explore the relationship between the SDM radius and the attention `beta` parameter.
6.  **Research Context & Extensions:** We will discuss the biological plausibility of SDM by mapping it to the cerebellar circuit.

#### **1.4 Learning Objectives & Estimated Time**

- **Objectives:**
  - Implement Sparse Distributed Memory (SDM) from the ground up.
  - Understand the mathematical properties of high-dimensional spaces that enable SDM.
  - Recreate the key insight of the lecture: show how SDM's circle intersection approximates the softmax function.
  - Gain a deeper, more fundamental understanding of *why* the Transformer attention mechanism is so effective.
  - Learn how SDM operations map to the neural circuitry of the cerebellum.
- **Estimated Time:** 90-120 minutes.

### Section 2: Mathematical Foundations

Before diving into the models themselves, we need to build the mathematical tools they rely on. The core of the lecture's argument is geometric, based on the behavior of vectors in high-dimensional spaces.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns

sns.set_theme(style="whitegrid")

def educational_hamming_distance(v1, v2):
    """
    Clear implementation of Hamming distance for understanding.
    - Based directly on the definition: count of differing elements.
    - Assumes binary vectors of the same length.
    - Extensive comments explaining each step.
    """
    # Ensure vectors are of the same length
    assert len(v1) == len(v2), "Vectors must be of the same length"
    
    distance = 0
    # Iterate through each element of the vectors
    for i in range(len(v1)):
        # If the bits at the current position are different, increment distance
        if v1[i] != v2[i]:
            distance += 1
    return distance

def optimized_hamming_distance(v1, v2):
    """
    Efficient implementation of Hamming distance.
    - Vectorized operations using numpy.
    - Assumes numpy arrays.
    """
    return np.count_nonzero(v1 != v2)

# --- Cosine Similarity ---
def educational_cosine_similarity(v1, v2):
    """
    Clear implementation of Cosine Similarity for understanding.
    - Follows the mathematical formula: (A . B) / (||A|| * ||B||)
    """
    dot_product = sum(a*b for a, b in zip(v1, v2))
    norm_v1 = sum(a*a for a in v1) ** 0.5
    norm_v2 = sum(b*b for b in v2) ** 0.5
    
    # Handle zero vectors to avoid division by zero
    if norm_v1 == 0 or norm_v2 == 0:
        return 0
        
    return dot_product / (norm_v1 * norm_v2)

def optimized_cosine_similarity(v1, v2):
    """
    Efficient implementation using numpy.
    """
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0:
        return 0
        
    return dot_product / (norm_v1 * norm_v2)

# --- Softmax ---
def educational_softmax(x, beta=1.0):
    """
    Clear implementation of the Softmax function.
    - Follows the formula: exp(beta * x_i) / sum(exp(beta * x_j))
    """
    # Exponentiate each element, scaled by beta
    exps = [np.exp(beta * i) for i in x]
    # Calculate the sum of all exponentiated elements
    sum_of_exps = sum(exps)
    # Normalize to get the softmax distribution
    softmax_dist = [j / sum_of_exps for j in exps]
    return softmax_dist

def optimized_softmax(x, beta=1.0):
    """
    Efficient implementation using numpy for numerical stability.
    - Subtracts max(x) before exponentiating to prevent overflow.
    """
    x = np.array(x)
    # For numerical stability, subtract the max value from each score
    e_x = np.exp(beta * (x - np.max(x)))
    return e_x / e_x.sum(axis=0)

# --- Testing the implementations ---
v_bin1 = np.random.randint(0, 2, 10)
v_bin2 = np.random.randint(0, 2, 10)
print(f"Vector 1 (binary): {v_bin1}")
print(f"Vector 2 (binary): {v_bin2}")
print(f"Educational Hamming Distance: {educational_hamming_distance(v_bin1, v_bin2)}")
print(f"Optimized Hamming Distance:   {optimized_hamming_distance(v_bin1, v_bin2)}\n")

v_cont1 = np.random.randn(10)
v_cont2 = np.random.randn(10)
print(f"Vector 1 (continuous): {np.round(v_cont1, 2)}")
print(f"Vector 2 (continuous): {np.round(v_cont2, 2)}")
print(f"Educational Cosine Similarity: {educational_cosine_similarity(v_cont1, v_cont2):.4f}")
print(f"Optimized Cosine Similarity:   {optimized_cosine_similarity(v_cont1, v_cont2):.4f}\n")

scores = np.array([1, 2, 5, 3, 1])
print(f"Original scores: {scores}")
print(f"Educational Softmax (beta=1): {np.round(educational_softmax(scores), 3)}")
print(f"Optimized Softmax (beta=1):   {np.round(optimized_softmax(scores), 3)}")
print(f"Optimized Softmax (beta=3):   {np.round(optimized_softmax(scores, beta=3), 3)}")

#### **Interactive Visualization: The Effect of `beta` on Softmax**
The lecture emphasizes that the softmax function makes distributions "peakier," effectively forcing the model to concentrate on the most relevant items. The `beta` parameter (often incorporated into the dot product scaling in Transformers) controls this behavior. A higher `beta` leads to a sharper, more concentrated distribution, while a lower `beta` results in a softer, more uniform one.

In [ ]:
def interactive_softmax_explorer():
    """
    Interactive widget for exploring the beta parameter in the softmax function.
    """
    initial_scores = np.array([0.5, 1.5, 4.0, 2.0, 0.8])

    def plot_softmax(beta, scores_text):
        try:
            scores = np.array([float(s) for s in scores_text.split(',')])
        except ValueError:
            print("Invalid input. Please provide comma-separated numbers.")
            return
        
        softmax_output = optimized_softmax(scores, beta=beta)
        
        plt.figure(figsize=(10, 5))
        
        # Plot original scores
        plt.subplot(1, 2, 1)
        plt.bar(range(len(scores)), scores, color='skyblue')
        plt.title('Original Scores')
        plt.xlabel('Item Index')
        plt.ylabel('Score')
        plt.ylim(0, np.max(scores) * 1.2 if np.max(scores) > 0 else 1)

        # Plot softmax distribution
        plt.subplot(1, 2, 2)
        plt.bar(range(len(softmax_output)), softmax_output, color='salmon')
        plt.title(f'Softmax Distribution (beta = {beta:.2f})')
        plt.xlabel('Item Index')
        plt.ylabel('Probability')
        plt.ylim(0, 1.1)

        plt.tight_layout()
        plt.show()
    
    beta_slider = widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Beta (β):')
    scores_input = widgets.Text(value=','.join(map(str, initial_scores)), description='Scores:', continuous_update=False)
    
    widgets.interactive(plot_softmax, beta=beta_slider, scores_text=scores_input)

# Run the interactive explorer
interactive_softmax_explorer()

### Section 3: Foundational Model: Sparse Distributed Memory

Sparse Distributed Memory (SDM) is an associative memory model proposed by Pentti Kanerva in 1988. It's designed to mimic properties of human memory, such as being robust to noisy inputs and having a massive capacity. It operates in a high-dimensional vector space.

- **Address Space:** A vast, high-dimensional binary space (e.g., {0, 1}<sup>1000</sup>).
- **Hard Locations:** A smaller, fixed set of randomly chosen addresses from this space, which represent the physical "neurons" of the memory.
- **Write Operation:** To store a data pattern (a vector) at a target address, you activate all *hard locations* within a certain Hamming distance (the "write radius") of the target address. The data pattern is then added to the contents of each activated hard location.
- **Read Operation:** To retrieve a pattern, you provide a query address. This activates all hard locations within a "read radius." The contents of these activated locations are summed (or averaged) together. This pooled result is the retrieved pattern. If the query is close to an address where a pattern was written, the retrieved pattern will be a close match to the original data.

In [ ]:
class SparseDistributedMemory:
    def __init__(self, num_hard_locations, dimensions, radius):
        """
        Initializes the Sparse Distributed Memory.
        - num_hard_locations: The number of 'neurons' in the memory.
        - dimensions: The dimensionality of the vector space.
        - radius: The Hamming distance for activating hard locations.
        """
        self.dimensions = dimensions
        self.radius = radius
        
        # Initialize hard locations with random binary vectors
        self.hard_locations = np.random.randint(0, 2, size=(num_hard_locations, dimensions))
        
        # Initialize memory contents (counters) to zero
        # These will store the superposition of patterns
        self.memory_contents = np.zeros((num_hard_locations, dimensions), dtype=int)
        print(f"Initialized SDM with {num_hard_locations} hard locations in {dimensions}-D space.")
        print(f"Activation radius (Hamming distance): {radius}")
        
    def _get_activated_indices(self, address):
        """Helper to find indices of hard locations within the radius of an address."""
        distances = np.array([optimized_hamming_distance(address, hl) for hl in self.hard_locations])
        return np.where(distances <= self.radius)[0]
        
    def educational_write(self, write_address, data_pattern):
        """
        Clear implementation of the SDM write operation.
        - Based directly on the SDM description.
        - Extensive comments explaining each step.
        - Uses a bipolar representation (+1, -1) for summation.
        """
        assert len(write_address) == self.dimensions and len(data_pattern) == self.dimensions
        
        # Convert from {0, 1} to {-1, 1} for superposition (summation)
        # This is a common way to implement the summation in SDM
        bipolar_pattern = 2 * data_pattern - 1
        
        # 1. Find all hard locations within the write radius of the write_address
        activated_indices = self._get_activated_indices(write_address)
        
        # 2. Add the data pattern to the memory contents of each activated neuron
        if len(activated_indices) > 0:
            self.memory_contents[activated_indices, :] += bipolar_pattern
        
        print(f"Writing pattern. Activated {len(activated_indices)} hard locations.")
        return activated_indices # Return for visualization

    def educational_read(self, query_address):
        """
        Clear implementation of the SDM read operation.
        - Retrieves and aggregates data from nearby neurons.
        """
        assert len(query_address) == self.dimensions
        
        # 1. Find all hard locations within the read radius of the query_address
        activated_indices = self._get_activated_indices(query_address)
        
        if len(activated_indices) == 0:
            print("Reading pattern. No hard locations activated.")
            return np.zeros(self.dimensions)
        
        # 2. Sum the contents of the activated hard locations
        summed_contents = np.sum(self.memory_contents[activated_indices, :], axis=0)
        
        # 3. Map back to binary space by thresholding (sign function)
        # If sum > 0, map to 1. If sum <= 0, map to 0.
        retrieved_pattern = (summed_contents > 0).astype(int)
        
        print(f"Reading pattern. Activated {len(activated_indices)} hard locations.")
        return retrieved_pattern, activated_indices

# --- Demonstration ---
DIMS = 128
LOCATIONS = 10000
RADIUS = 50

sdm = SparseDistributedMemory(num_hard_locations=LOCATIONS, dimensions=DIMS, radius=RADIUS)

# Create three distinct patterns to store
pattern_A_addr = np.random.randint(0, 2, DIMS)
pattern_A_data = np.copy(pattern_A_addr) # For simplicity, address is the data

pattern_B_addr = np.random.randint(0, 2, DIMS)
pattern_B_data = np.copy(pattern_B_addr)

# Write patterns into memory
print("\n--- Writing Data ---")
sdm.educational_write(pattern_A_addr, pattern_A_data)
sdm.educational_write(pattern_B_addr, pattern_B_data)

# --- Reading Data ---
print("\n--- Reading Data ---")
# Query with the original address
retrieved_A, _ = sdm.educational_read(pattern_A_addr)
print(f"Querying with original pattern A. Distance from original: {optimized_hamming_distance(retrieved_A, pattern_A_data)}")

# Create a noisy version of pattern A
noise_level = 15
noise_indices = np.random.choice(DIMS, noise_level, replace=False)
noisy_pattern_A = np.copy(pattern_A_addr)
noisy_pattern_A[noise_indices] = 1 - noisy_pattern_A[noise_indices] # Flip bits
print(f"\nCreated a noisy version of pattern A with {noise_level} flipped bits.")

# Query with the noisy address
retrieved_A_from_noisy, _ = sdm.educational_read(noisy_pattern_A)
print(f"Querying with noisy pattern A. Distance from original: {optimized_hamming_distance(retrieved_A_from_noisy, pattern_A_data)}")
print("Notice how the retrieved pattern is much closer to the original than the noisy query was.")

### Section 4: Core Research Content: Attention Approximates SDM

This is the central thesis of the lecture. The connection between Transformer Attention and SDM is not in the write mechanism, but in the **read mechanism**. Let's break down the analogy:

| Transformer Attention Component | Sparse Distributed Memory Analogue |
| :--- | :--- |
| Query vector (`q`) | The `query_address` used to read from memory. |
| Key vectors (`K`) | The original `write_address` locations of stored patterns. |
| Value vectors (`V`) | The `data_pattern` that was stored at each `write_address`. |
| `softmax(q • K)` | The **normalized count of neurons in the intersection of hyperspheres**. |
| Weighted sum of `V` | The weighted sum of `data_patterns` retrieved from activated neurons. |

The key insight is that the number of neurons in the intersection between a query's read-hypersphere and a key's write-hypersphere decays **approximately exponentially** as the distance between the query and key increases. This exponential decay is precisely what the softmax function computes.

#### **Visualizing the Core Claim: Hypersphere Intersection vs. Softmax**

Let's now reproduce the central experiment from the lecture. We will simulate an SDM and plot the number of neurons in the 'circle intersection' as we move a query vector away from a key vector. We will then overlay a fitted exponential curve (representing the softmax) to show the close approximation.

In [ ]:
def interactive_sdm_attention_approximation():
    """
    Interactive widget to visualize the approximation of softmax by SDM's
    circle intersection count.
    """
    
    @widgets.interact(
        log_scale=widgets.Checkbox(value=True, description='Log Scale Y-Axis'),
        dims=widgets.IntSlider(value=256, min=64, max=1024, step=64, description='Dimensions:'),
        num_locations=widgets.IntSlider(value=20000, min=5000, max=50000, step=5000, description='Neurons:'),
        radius=widgets.IntSlider(value=100, min=20, max=200, step=5, description='Radius:')
    )
    def run_simulation(log_scale, dims, num_locations, radius):
        # 1. Setup SDM
        hard_locations = np.random.randint(0, 2, size=(num_locations, dims))
        
        # 2. Define a fixed Key (original write address) and a moving Query
        key_address = np.zeros(dims, dtype=int)
        
        intersection_counts = []
        distances = []

        # 3. Move the Query away from the Key, bit by bit
        max_dist = int(dims * 0.4)
        for dist in range(0, max_dist, 2):
            distances.append(dist)
            query_address = np.copy(key_address)
            if dist > 0:
                query_address[:dist] = 1 # Flip 'dist' bits to create distance
            
            # 4. Calculate the size of the circle intersection
            # This is the number of neurons activated by BOTH the key and the query
            key_distances = np.sum(np.abs(hard_locations - key_address), axis=1)
            key_activations = key_distances <= radius
            
            query_distances = np.sum(np.abs(hard_locations - query_address), axis=1)
            query_activations = query_distances <= radius
            
            intersection = np.sum(key_activations & query_activations)
            intersection_counts.append(intersection)

        intersection_counts = np.array(intersection_counts)
        distances = np.array(distances)
        
        # Normalize counts to look like probabilities
        normalized_counts = intersection_counts / np.max(intersection_counts) if np.max(intersection_counts) > 0 else intersection_counts

        # 5. Fit an exponential curve (y = exp(-beta * x)) to the data
        # This is equivalent to softmax where similarity is negative distance
        # We use a simple log-linear regression to find the 'beta'
        valid_indices = normalized_counts > 0
        if np.sum(valid_indices) > 1:
            log_counts = np.log(normalized_counts[valid_indices])
            # The 'beta' here is our effective softmax scaling factor
            beta = -np.polyfit(distances[valid_indices], log_counts, 1)[0]
            
            # Generate the softmax/exponential curve using cosine similarities mapped from Hamming distances
            cosine_sims = 1 - (2 * distances) / dims
            softmax_approx = optimized_softmax(cosine_sims * dims, beta=beta) # Scale by dims to get a good range
            softmax_approx /= np.max(softmax_approx) # Normalize to match the plot
        else:
            beta = 0
            softmax_approx = np.zeros_like(normalized_counts)
        
        # 6. Plot the results
        plt.figure(figsize=(10, 6))
        plt.plot(distances, normalized_counts, 'o-', label='SDM Intersection Count (Normalized)', color='blue')
        if np.sum(valid_indices) > 1:
            plt.plot(distances, softmax_approx, 'r--', label=f'Softmax/Exponential Fit (β ≈ {beta:.3f})')
            
        plt.title('SDM Circle Intersection Decays Exponentially', fontsize=16)
        plt.xlabel('Hamming Distance between Key and Query', fontsize=12)
        plt.ylabel('Normalized Intersection Size', fontsize=12)
        if log_scale:
            plt.yscale('log')
            plt.ylabel('Normalized Intersection Size (Log Scale)', fontsize=12)
        
        plt.legend()
        plt.grid(True, which="both", ls="--")
        plt.show()

interactive_sdm_attention_approximation()

**Observation:** As you can see in the plot above, the number of neurons in the circle intersection (blue line) follows a clear exponential decay, which is closely matched by the fitted softmax curve (red dashed line). When viewed on a log scale, this exponential relationship becomes a straight line, confirming the approximation. This provides strong evidence for the lecture's core claim.

### Section 5: Experimental Analysis

#### **5.1 Reproducing Experiment: Optimal Beta Coefficients**

The lecture presented evidence that in Transformers where the `beta` coefficient of the softmax is learned, the learned values correspond to optimal configurations of SDM for different tasks (e.g., maximizing memory capacity or robustness to noise).

While we cannot train a full Transformer here, we can simulate the result. We will generate a hypothetical distribution of learned `beta` values and plot them against the theoretical optimal `beta` values for SDM derived from different objectives. This demonstrates that if Attention is indeed implementing SDM, its learned parameters should fall within a range predicted by SDM theory.

In [ ]:
def plot_beta_distribution():
    """
    Reproduces the conceptual plot from the lecture showing the distribution
    of learned beta coefficients against optimal SDM betas.
    """
    # Theoretical optimal betas for SDM under different assumptions
    # These are illustrative values based on the lecture's figure.
    sdm_optimal_betas = {
        'Max Memory Capacity': 0.35,
        'Max Info Content': 0.65,
        'Max Query Noise': 0.95
    }

    # Simulate a distribution of 'learned' beta coefficients from a hypothetical model
    # The distribution is skewed towards the 'Max Query Noise' value, as noted in the lecture.
    np.random.seed(42)
    learned_betas = np.random.normal(loc=0.85, scale=0.2, size=1000)
    learned_betas = np.clip(learned_betas, 0.1, 1.5) # Ensure they fall in a reasonable range

    plt.figure(figsize=(12, 7))
    sns.histplot(learned_betas, bins=30, kde=True, label='Learned β Distribution (Simulated)')

    colors = ['r', 'g', 'purple']
    for i, (label, beta_val) in enumerate(sdm_optimal_betas.items()):
        plt.axvline(x=beta_val, color=colors[i], linestyle='--', linewidth=2, label=f'Optimal SDM β for {label}')

    plt.title('Distribution of Learned Attention β Coefficients vs. Optimal SDM βs', fontsize=16)
    plt.xlabel('Beta (β) Value', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend()
    plt.show()

plot_beta_distribution()

**Analysis:** This plot illustrates the lecture's finding. The distribution of `beta` values that a real Transformer might learn tends to cluster around the SDM configuration that is most robust to noisy queries. This makes intuitive sense for real-world data, which is inherently noisy and complex. It suggests the Transformer isn't just learning random scaling factors, but rather it's converging on a parameterization that reflects a fundamental trade-off in associative memory systems.

### Section 6: Research Context & Extensions

#### **6.1 Biological Plausibility: Mapping SDM to the Cerebellum**

A compelling aspect of this research is that SDM has a plausible implementation in the neural circuits of the cerebellum. The unique wiring of this brain region provides a substrate for the key operations of SDM. The lecture breaks down this mapping as follows:

| SDM Component/Operation | Cerebellar Analogue | Function |
| :--- | :--- | :--- |
| High-dimensional space | **Granule Cells** | A massive number of granule cells (~70% of all neurons in the brain) create a high-dimensional expansion of inputs. |
| Hard Locations (Neurons) | **Granule Cells** | Each granule cell acts as a "hard location" with a fixed address determined by its inputs. |
| Input Address (Query/Key) | **Mossy Fibers** | These are the main inputs to the cerebellum, carrying context and sensory information. Their pattern of activation represents the input address. |
| Stored Value (Data) | **Climbing Fibers** | A separate, powerful input that wraps around a Purkinje cell. It provides a strong "teaching" or "value" signal. This separation is analogous to having distinct Keys (from Mossy Fibers) and Values (from Climbing Fibers). |
| Summation/Readout | **Purkinje Cells** | Each Purkinje cell receives input from thousands of granule cells (via parallel fibers) and performs a summation, analogous to the SDM read operation. |

This mapping suggests that the core computation of Transformer Attention—a weighted sum based on similarity—is not an abstract invention but may be a fundamental cognitive operation refined by evolution and implemented in our own brains.

#### **6.2 Research Questions and Future Directions**

This work opens up several profound questions that bridge neuroscience and AI:

1.  **Is the Transformer's success due to its implementation of a core cognitive function?**
    The parallel between attention and a biologically plausible memory model suggests that Transformers may have inadvertently rediscovered an efficient algorithm for associative memory and information retrieval that the brain already uses.

2.  **Is SDM the correct theory for cerebellar function?**
    The empirical success of Transformers lends new weight to SDM as a leading theory for what the cerebellum does. If the algorithm is powerful enough to drive state-of-the-art AI, it is a strong candidate for a key neural computation.

3.  **Can we use this link to improve future architectures?**
    Understanding the principles of SDM could help us move beyond heuristic improvements to Transformers. For instance, exploring other aspects of SDM, such as its capacity for continual learning or its relationship with vector symbolic architectures, might lead to new and more powerful AI models.